In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import os

In [2]:
#from factory.LstmDV import LstmDV as D_VECTOR
from factory.MetaDV import MetaDV as D_VECTOR
from util.Dataloader import generate_dataset

In [3]:
num_uttrs = 200
len_crop = 256
batch_size = 32
num_classes=80
use_shuffle = True
save_name = "metadv_vctk80.pt"

In [4]:
d_vector = D_VECTOR(num_classes,len_crop).to('cuda:0')
optimizer = torch.optim.Adam(d_vector.parameters(), 1e-4) 
criterion = nn.CrossEntropyLoss()

In [5]:
train_loader = generate_dataset(rootDir='train_spmel_vctk',len_crop=len_crop,num_uttrs=num_uttrs, batch_size=batch_size,use_shuffle=use_shuffle)

Found directory: train_spmel_vctk
Processing speaker: p225
Processing speaker: p226
Processing speaker: p227
Processing speaker: p228
Processing speaker: p229
Processing speaker: p230
Processing speaker: p231
Processing speaker: p232
Processing speaker: p233
Processing speaker: p234
Processing speaker: p236
Processing speaker: p237
Processing speaker: p238
Processing speaker: p239
Processing speaker: p240
Processing speaker: p241
Processing speaker: p243
Processing speaker: p244
Processing speaker: p245
Processing speaker: p246
Processing speaker: p247
Processing speaker: p248
Processing speaker: p249
Processing speaker: p250
Processing speaker: p251
Processing speaker: p252
Processing speaker: p253
Processing speaker: p254
Processing speaker: p255
Processing speaker: p256
Processing speaker: p257
Processing speaker: p258
Processing speaker: p259
Processing speaker: p260
Processing speaker: p261
Processing speaker: p262
Processing speaker: p263
Processing speaker: p264
Processing speak

In [6]:
epochs = 50

In [7]:
early_stop_count = 0
early_repeate = 3
last_loss = 100.0
for epoch in range(epochs):
    n_correct,n_total,train_loss,train_acc,counter = 0,0,0,0,0
    for data in train_loader:
        inputs, targets = data
        targets = targets.view(-1).long()
        current_sample = inputs.size(0)  
        output,_ = d_vector(inputs)
        n_correct += (torch.max(output, 1)[1].long().view(targets.size()) == targets).sum().item()
        n_total += current_sample
        train_acc_temp = 100. * n_correct / n_total
        loss = criterion(output, targets)
        train_loss += loss.item()
        train_acc += train_acc_temp
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        counter += 1
    if epoch == 0:
        last_loss = (train_loss/counter)
    elif last_loss > (train_loss/counter):
        last_loss =  train_loss/counter
        early_stop_count = 0
    else:
        last_loss =  train_loss/counter
        early_stop_count += 1
        
        
    print(f"AVE LOSS --- {train_loss/counter}")
    print(f"ACC --- {train_acc/counter}")
    
    if early_stop_count == early_repeate:
        print("*************************")
        print(f" Training Stop at epoch --- {epoch}")
        print(f" Final AVE LOSS --- {train_loss/counter}")
        print(f" Final ACC --- {train_acc/counter}")
        print("*************************")
        break

AVE LOSS --- 4.118798948736751
ACC --- 2.2040555222532277
AVE LOSS --- 1.8934988835278679
ACC --- 30.716949833297733
AVE LOSS --- 0.7468184788437451
ACC --- 74.9113562585501
AVE LOSS --- 0.42847216269549204
ACC --- 86.16139521049473
AVE LOSS --- 0.377702023088932
ACC --- 88.57798110218323
AVE LOSS --- 0.26902272501850827
ACC --- 91.4271021761521
AVE LOSS --- 0.27391697839998147
ACC --- 91.36986135837284
AVE LOSS --- 0.24939728730960803
ACC --- 92.72293561410983
AVE LOSS --- 0.2124516866991625
ACC --- 92.78148289517874
AVE LOSS --- 0.2127936525927747
ACC --- 93.7706338691634
AVE LOSS --- 0.1619911240413785
ACC --- 95.49333102356934
AVE LOSS --- 0.19704837337245837
ACC --- 93.78504517602873
AVE LOSS --- 0.160605061519891
ACC --- 95.21050977101083
AVE LOSS --- 0.1590234730283127
ACC --- 95.75974153014157
AVE LOSS --- 0.16342210717608824
ACC --- 94.403335851907
AVE LOSS --- 0.14702841981368905
ACC --- 95.08304406655694
AVE LOSS --- 0.16443427308079075
ACC --- 95.6521087491015
AVE LOSS --- 

In [8]:
torch.save(d_vector,save_name)